# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Notebooks/task_ESRGAN

[Errno 2] No such file or directory: '/content/drive/MyDrive/Notebooks/task_ESRGAN'
/content


In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 755, done.
remote: Total 755 (delta 0), reused 0 (delta 0), pack-reused 755
Receiving objects: 100% (755/755), 5.37 MiB | 6.38 MiB/s, done.
Resolving deltas: 100% (410/410), done.


In [ ]:
%cd Real-ESRGAN

/content/drive/MyDrive/Notebooks/task_ESRGAN/Real-ESRGAN


In [ ]:
%cd experiments/pretrained_models

/content/drive/MyDrive/Notebooks/task_ESRGAN/Real-ESRGAN/experiments/pretrained_models


In [ ]:
!wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth

In [ ]:
%cd /content/drive/MyDrive/Notebooks/task_ESRGAN/Real-ESRGAN/
!pip install -r requirements.txt

In [ ]:
%cd /content/drive/MyDrive/Notebooks/task_ESRGAN

/content/drive/MyDrive/Notebooks/task_ESRGAN


In [ ]:
%ls

enhanced_frames/  input_videos/  Real-ESRGAN/


# Convert Video into Frames

In [ ]:
import cv2
from tqdm import tqdm
import os

# Set the paths in Google Colab
video_frames_path = "/content/drive/MyDrive/Notebooks/task_ESRGAN/video_frames"
input_videos_path = "/content/drive/MyDrive/Notebooks/task_ESRGAN/input_videos"

# Create the video_frames directory if it doesn't exist
os.makedirs(video_frames_path, exist_ok=True)

for video_name in os.listdir(input_videos_path):
    video_path = os.path.join(input_videos_path, video_name)
    frame_path = os.path.join(video_frames_path, os.path.splitext(video_name)[0])

    # Create the frame directory if it doesn't exist
    os.makedirs(frame_path, exist_ok=True)

    video_cap = cv2.VideoCapture(video_path)
    frame_count = int(video_cap.get(cv2.CAP_PROP_FRAME_COUNT))

    for i in tqdm(range(frame_count)):
        is_success, frames = video_cap.read()
        if not is_success:
            break  # Break the loop if there are no more frames

        path_frame = os.path.join(frame_path, f"{i}.jpg")
        cv2.imwrite(path_frame, frames)

    video_cap.release()


100%|██████████| 1848/1848 [01:11<00:00, 25.84it/s]


# Feed Each Frame to Model

In [ ]:
%cd /content/drive/MyDrive/Notebooks/task_ESRGAN/Real-ESRGAN

/content/drive/MyDrive/Notebooks/task_ESRGAN/Real-ESRGAN


In [ ]:
import os
splitted_frame_path = "/content/drive/MyDrive/Notebooks/task_ESRGAN/video_frames"
enhanced_frame_path = "/content/drive/MyDrive/Notebooks/task_ESRGAN/enhanced_frames"
os.makedirs(enhanced_frame_path,exist_ok = True)

In [ ]:
for dir in os.listdir(splitted_frame_path):
  for frames in os.listdir(os.path.join(splitted_frame_path,dir)):
    input_path = os.path.join(splitted_frame_path,dir,frames)
    output_path = os.path.join(enhanced_frame_path,dir)
      !python inference_realesrgan.py --model_name RealESRGAN_x4plus -i $input -o $output --outscale 4 --face_enhance --fp32 -dn 1

# Convert Enhanced Frames into Videos

In [ ]:
import os
import cv2
from tqdm import tqdm
from pathlib import Path


def merge_frames_to_videos():
    input_video_path = "input_videos/"
    enhanced_frame_path = "enhanced_frames/"
    out_path = "output_videos"
    os.makedirs(out_path,exist_ok=True)

    for frame_dir in os.listdir(enhanced_frame_path):

        video_path = os.path.join(input_video_path,f"{frame_dir}.mp4")
        video_capture = cv2.VideoCapture(video_path)
        fps = video_capture.get(cv2.CAP_PROP_FPS)  ## Gets FPS from Original Video files

        list_image_files = os.listdir(os.path.join(enhanced_frame_path,frame_dir))
        list_image_files = sorted(list_image_files,key = lambda x: int(x.split("_")[0]))  ## sorting all files
        print(list_image_files)

        merge_images = []
        for image_file in tqdm(list_image_files):
            image = cv2.imread(f"{enhanced_frame_path}/{frame_dir}/{image_file}")
            merge_images.append(image)
        # print(merge_images)

        output_path = os.path.join(out_path,f"{frame_dir}.mp4")
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        video_writer = cv2.VideoWriter(output_path,fourcc,fps,(merge_images[0].shape[1],merge_images[0].shape[0]))

        for image in tqdm(merge_images):
            video_writer.write(image)

        video_writer.release()
        cv2.destroyAllWindows()

if __name__ =="__main__":
    merge_frames_to_videos()